In [68]:
import pandas as pd
import time
from collections import defaultdict, Counter

In [65]:
g = pd.read_csv("data/attendances_over_time.csv.gz", encoding='latin-1', 
                header=None, names=["cust_id","age_gr","mtype_sec", "n_attend"])

In [66]:
g.head()

,cust_id,age_gr,mtype_sec,n_attend
0,11168,50-54,RUNION,1
1,11171,50-54,RUNION,1
2,11179,60-64,FOLK,1
3,11179,60-64,AFL,5
4,11181,55-59,RUNION,24


In [ ]:
g_purch_byage = g.groupby(["cust_id", "age_gr"])["mtype_sec"].apply(list)  # obtain a Series wit Multi Index

In [45]:
age_gr_each_customer = g.groupby(["cust_id"])["age_gr"].apply(set).apply(list)  # note: sorting is done by groupby

In [13]:
g1

cust_id  age_gr
11168    50-54                               [RUNION]
11171    50-54                               [RUNION]
11179    60-64                            [FOLK, AFL]
11181    55-59                               [RUNION]
11196    30-34                               [ALTERN]
         35-39                                  [AFL]
11197    50-54                               [RUNION]
11206    50-54                               [RUNION]
11228    40-44                       [ALTERN, LEAGUE]
11249    50-54                               [RUNION]
11277    50-54                    [ROCK, RUNION, POP]
11284    40-44                               [HIPHOP]
11285    45-49     [RUNION, CARRACE, 80SROCK, ALTERN]
Name: mtype_sec, dtype: object

In [46]:
age_gr_each_customer.head()

cust_id
11168           [50-54]
11171           [50-54]
11179           [60-64]
11181           [55-59]
11196    [30-34, 35-39]
Name: age_gr, dtype: object

In [33]:
last_known_age_gr = g.iloc[:20].groupby(["cust_id"])["age_gr"].apply(max)  # obtain a Series wit Multi Index
youngest_age_gr = g.iloc[:20].groupby(["cust_id"])["age_gr"].apply(min)  # obtain a Series wit Multi Index

In [34]:
youngest_age_gr

cust_id
11168    50-54
11171    50-54
11179    60-64
11181    55-59
11196    30-34
11197    50-54
11206    50-54
11228    40-44
11249    50-54
11277    50-54
11284    40-44
11285    45-49
Name: age_gr, dtype: object

In [62]:
cust_longterm = age_gr_each_customer.loc[age_gr_each_customer.apply(lambda x: True if len(x) > 1 else False)].index.values

In [55]:
cust_longterm.head()

cust_id
11196    [30-34, 35-39]
11391    [60-64, 65-69]
11392    [55-59, 60-64]
11485    [45-49, 50-54]
11581    [50-54, 55-59]
Name: age_gr, dtype: object

In [59]:
cust_shortterm = age_gr_each_customer.loc[age_gr_each_customer.apply(lambda x: True if len(x) == 1 else False)].index.values

In [60]:
len(cust_shortterm)

4494603

In [64]:
feat_sht_customers = pd.get_dummies(d[d.cust_id.isin(cust_shortterm), ["mtype_sec","age_gr"]], columns=["mtype_sec"])
feat_sht_customers.head()

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import LabelEncoder

In [18]:
X_train, X_test, y_train, y_test = train_test_split(d1.drop("age_gr", axis=1), d1["age_gr"], 
                                                    test_size=0.3, random_state=94, stratify=d1["age_gr"])

In [19]:
print("created training data - features {} and target {}".format(X_train.shape, y_train.shape))

created training data - features (13026316, 45) and target (13026316,)


In [23]:
gnb = BernoulliNB()
gnb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

print("accuracy: {:.3f}".format(accuracy_score(y_test, svc.predict(X_test))))
scores = cross_val_score(svc, X_train, y_train, cv=5)


#print("accuracy: {:.3f}".format(accuracy_score(y_test, gnb.predict(X_test))))
# scores = cross_val_score(gnb, X_train, y_train, cv=5)
print("cross-validation scores: {}, {}, {}, {}, {}".format(*scores))